In [1]:
from abc import abstractmethod
from tabicl.prior.dataset import SCMPrior
from tabicl.prior.prior_config import DEFAULT_FIXED_HP, DEFAULT_SAMPLED_HP

import numpy as np
from scipy.stats import loguniform
import joblib
import math

import torch
import torch.nn.functional as F
from torch import Tensor
from torch.nested import nested_tensor
from torch.utils.data import IterableDataset

from typing import Optional, Tuple, Dict, Any, Union

from mcpfn.prior.dataset import DummyPrior, MCARPrior
    
mcar_prior = MCARPrior(num_missing=10, max_features = 20, max_seq_len=100)

X, y, d, seq_lens, train_sizes = mcar_prior.get_batch(batch_size = 4)

print(X.shape)
print(seq_lens, train_sizes)

# import matplotlib.pyplot as plt
# import seaborn as sns
# sns.heatmap(torch.isnan(X), cbar=False, cmap='Blues')

KeyboardInterrupt: 

In [3]:
from mcpfn.interface import ImputePFN
import numpy as np

imputer = ImputePFN(device='cpu', 
                    encoder_path='./src/mcpfn/model/encoder.pth',
                    borders_path='./borders.pt',
                    checkpoint_path='./stage1/checkpoint/test.ckpt')

X = np.random.rand(5, 5)
print(X)
X[np.random.rand(*X.shape) < 0.1] = np.nan
print(X)

out = imputer.impute(X)
print(out)

[[0.85147824 0.94982095 0.01447291 0.88739215 0.92224733]
 [0.84816912 0.96909181 0.76627075 0.1521822  0.9371314 ]
 [0.62213287 0.32389656 0.51246714 0.16060332 0.07107045]
 [0.88621993 0.94509748 0.3024038  0.90494541 0.14689113]
 [0.99369417 0.80505457 0.75710559 0.56601662 0.04271451]]
[[0.85147824 0.94982095 0.01447291 0.88739215 0.92224733]
 [0.84816912 0.96909181 0.76627075 0.1521822  0.9371314 ]
 [0.62213287 0.32389656 0.51246714 0.16060332        nan]
 [0.88621993 0.94509748 0.3024038  0.90494541 0.14689113]
 [0.99369417 0.80505457 0.75710559 0.56601662        nan]]
[[0.85147824 0.94982095 0.01447291 0.88739215 0.92224733]
 [0.84816912 0.96909181 0.76627075 0.1521822  0.9371314 ]
 [0.62213287 0.32389656 0.51246714 0.16060332 0.6657229 ]
 [0.88621993 0.94509748 0.3024038  0.90494541 0.14689113]
 [0.99369417 0.80505457 0.75710559 0.56601662 0.66687705]]


In [ ]:


# Test matrix of size 10 x 10 with 10% missing values
X = np.random.rand(10, 10)

# Set 10% of values to NaN
X[np.random.rand(*X.shape) < 0.1] = np.nan

print(X)

# Impute the missing values
out = imputer.impute(X)
print(out)

[[       nan 0.26508802 0.45874861 0.0941904         nan        nan
  0.78678157        nan 0.24154306 0.966167  ]
 [0.7120501  0.62928611 0.38961149 0.84581834 0.68264313 0.71990588
  0.05563086 0.07735007 0.52725047 0.23147827]
 [0.85064097 0.82042472 0.18773755        nan        nan 0.89968949
  0.77239313 0.75011495 0.25972265 0.90797397]
 [0.21911165        nan 0.11899991 0.6578947  0.23433034 0.66283502
  0.39087968 0.15767099 0.74751069 0.27074889]
 [0.9097427  0.72222151 0.34162459 0.21494868 0.6272933  0.22539053
  0.09955998 0.76165163 0.14568295        nan]
 [0.58476252 0.62025765        nan 0.27881058 0.06172408 0.2059866
  0.89020498 0.58859036 0.2019589  0.16058151]
 [0.38475496 0.63122182 0.85924915 0.64069642 0.81801942 0.89867385
  0.30907909 0.30068318 0.55514978 0.23822882]
 [0.62506987 0.94016654 0.98494254 0.92067309 0.58940561 0.97521641
         nan 0.78120638 0.74183974 0.88417763]
 [0.48093479 0.69123396 0.51658004 0.22068094 0.957913          nan
  0.80547197 

In [4]:
out = imputer.impute(X)
print(out)

[[5.54466416 8.         3.        ]
 [4.         8.         6.        ]
 [7.         8.         9.        ]]


In [5]:
import numpy as np

X = np.array([[np.nan, np.nan, 3], [4, np.nan, 6], [7, 8, 9]])

for i,j in zip(*np.where(np.isnan(X))):
    print(i,j)

0 0
0 1
1 1


In [ ]:
batch = torch.load('/mnt/volume_nyc2_1750872154987/data/small_mcar/val/batch_000000.pt')

In [2]:
X = batch['X']
y = batch['y']
d = batch['d']

In [1]:
from mcpfn.prior.genload import LoadPriorDataset
from tabpfn import TabPFNRegressor
import torch
import einops

val_dataset = LoadPriorDataset(
    data_dir="/Users/jfeit/tabular/mcpfn/data/val",
    batch_size=5,
    ddp_world_size=1,
    ddp_rank=0,
    start_from=0,
    delete_after_load=False,
    device='cpu',
)

X, y, d, seq_lens, train_sizes = next(iter(val_dataset))

from mcpfn.model.mcpfn import MCPFN

model = MCPFN(encoder_path="/Users/jfeit/tabular/mcpfn/src/mcpfn/model/encoder.pth")

y_train = y.clone()

# Create a mask for each row up to its train_size
mask = torch.zeros_like(y, dtype=torch.bool)
for i in range(len(train_sizes)):
    mask[i, :train_sizes[i]] = True

# Set values after train_size to nan for each row
y_train[~mask] = torch.nan

Using FlashAttention: False


In [9]:
from mcpfn.model.bar_distribution import FullSupportBarDistribution
import torch

bar_distribution = FullSupportBarDistribution(borders=torch.load('/Users/jfeit/tabular/mcpfn/borders.pt'))

In [14]:
pred = torch.randn(10, 5, 5000) # t b h
y = torch.randn(10, 5) # t b
train_sizes = torch.randint(1, 10, (5,))

In [32]:
loss = bar_distribution(logits=pred, y = y)

In [33]:
mask = torch.zeros_like(y, dtype=torch.bool)
for i in range(len(train_sizes)):
    mask[train_sizes[i]:, i] = True

loss[~mask] = torch.nan

loss

tensor([[   nan,    nan,    nan,    nan,    nan],
        [   nan,    nan,    nan,    nan,    nan],
        [   nan,    nan,    nan,    nan,    nan],
        [   nan, 3.2830, 3.9292,    nan,    nan],
        [   nan, 4.8321, 4.2667,    nan,    nan],
        [   nan, 4.4090, 3.0524,    nan,    nan],
        [   nan, 2.7097, 3.6088, 2.3133,    nan],
        [3.5272, 3.9375, 3.9571, 4.6207,    nan],
        [2.8377, 2.6702, 3.7146, 3.3269, 4.2096],
        [2.7557, 4.4679, 4.2730, 5.2302, 2.5469]])

In [2]:
out = model(X, y_train)

In [5]:
train_sizes

tensor([26, 33, 42, 27, 30])

In [4]:
out.shape

torch.Size([5, 50, 5000])

In [6]:
out.shape

torch.Size([50, 5, 5000])

In [8]:
train_out.shape

torch.Size([50, 5, 5000])

In [6]:
X_example, y_example = X[0,:,:].cpu(), y[0,:].cpu()

reg = TabPFNRegressor(device='cuda')
reg.fit(X_example, y_example)

TabPFNRegressor(device='cuda')

In [4]:
import einops

# Compute mean along dim=2 (last dimension), ignoring NaNs
mean_vals = torch.nanmean(X, dim=2, keepdim=True)  # shape: [1, 2000, 1]

# Find the NaNs
nan_mask = torch.isnan(X)  # shape: [1, 2000, 20]

# Expand mean_vals to match x's shape for indexing
mean_vals_expanded = mean_vals.expand_as(X)

# Replace NaNs with corresponding mean values
X[nan_mask] = mean_vals_expanded[nan_mask]

train_size = 40

y_train = y[:, :train_size]
y_test = y[:, train_size:]

In [7]:
model = reg.model_.to('cuda')

In [8]:
model.forward(X, y, single_eval_pos=40)

torch.float32 torch.float32 torch.float32
True
False


RuntimeError: CUDA error: invalid configuration argument
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [48]:
import numpy as np

for i in range(10):
    X_part = X[i,:,:].cpu()
    y_part = y[i,:].cpu()
    
    X_train = X_part[:40, :]
    X_test = X_part[40:, :]
    
    y_train_part = y_part[:40]
    y_test_part = y_part[40:]
    
    reg = TabPFNRegressor(device='cuda')
    reg.fit(X_train, y_train_part)
    
    pred = reg.predict(X_test)
    
    print('MAE: ', np.mean(np.abs(pred - y_test_part.cpu().numpy())))

TypeError: flash_attn_qkvpacked_func() got multiple values for argument 'dropout_p'

In [2]:
from mcpfn.prior.training_set_generation import TabICLSCMPrior

prior = TabICLSCMPrior(
    batch_size = 4,
    num_samples = 10,
    num_features = 5,
    num_missing = 10,
    device = 'cpu'
)

X_nested, y_nested, d, seq_lens, train_sizes = prior.get_batch()

In [6]:
import numpy as np

np.nanmax(np.array(X_nested)), np.nanmin(np.array(X_nested))

(5.433702, -2.46117)

In [4]:
178 / 100

1.78

In [15]:
train_sizes

tensor([40, 40, 40, 40])

In [16]:
print(X_nested[0].shape)

torch.Size([50, 13])


### Nested tensors

In [1]:
import torch

a = torch.randn(2, 6)
b = torch.randn(4, 6)

nested_a = torch.nested.nested_tensor([a, b], layout=torch.jagged)

print(nested_a)

NestedTensor(size=(2, j1, 6), offsets=tensor([0, 2, 6]), contiguous=True)


In [6]:
# Create tensors with different lengths in second dimension
c = torch.randn(2, 2, 4)
d = torch.randn(2, 5, 4)
e = torch.randn(2, 3, 4)

# Create nested tensor with jagged second dimension
nested_b = torch.nested.nested_tensor([c, d, e], layout=torch.jagged)

print(nested_b.shape)


torch.Size([3, 2, j3, 4])


In [1]:
from mcpfn.model.layers import OneHotAndLinearBarDistribution
from mcpfn.model.bar_distribution import get_bucket_limits

embed = OneHotAndLinearBarDistribution(borders = get_bucket_limits(num_outputs = 100, ys = y[0]), embed_dim = 10)

  0%|          | 0/1 [00:00<?, ?it/s]

No DDP training
Model has 32161800 parameters.
Automatic Mixed Precision is enabled.
No checkpoint found, starting from scratch.


In [7]:
from mcpfn.model.mcpfn import MCPFN

model_mcpfn = MCPFN(encoder_path = 'src/mcpfn/model/encoder.pth')
torch.compile(model_mcpfn)

ModuleNotFoundError: No module named 'mcpfn.model.embedding'

In [20]:
from mcpfn.model.tabicl import TabICL
import torch

model_tabicl = TabICL()
torch.compile(model_tabicl)
print(f'TabICL parameters: {sum(p.numel() for p in model_tabicl.parameters() if p.requires_grad)}')

from mcpfn.model.mcpfn import MCPFN

model_mcpfn = MCPFN(encoder_path = 'src/mcpfn/model/encoder.pth')
torch.compile(model_mcpfn)
print(f'MCPFN parameters: {sum(p.numel() for p in model_mcpfn.parameters() if p.requires_grad)}')

RuntimeError: Class values must be non-negative.

In [2]:
X, y = torch.randn(1, 2000, 10), torch.randn(1, 2000)
X_test = torch.randn(1, 10, 10)

In [3]:
%load_ext snakeviz

In [12]:
%%snakeviz -t

out = model_tabicl.forward(torch.cat([X, X_test], dim=1), y, d=torch.tensor([10]))

 
*** Profile stats marshalled to file '/var/folders/zc/6nw_r5b13ws3wfcpdxz6bzcw0000gn/T/tmpa43dxoc6'.
Opening SnakeViz in a new tab...
snakeviz web server started on 127.0.0.1:8080; enter Ctrl-C to exit
http://127.0.0.1:8080/snakeviz/%2Fvar%2Ffolders%2Fzc%2F6nw_r5b13ws3wfcpdxz6bzcw0000gn%2FT%2Ftmpa43dxoc6


In [4]:
%%snakeviz -t

out = model_mcpfn.forward(torch.cat([X, X_test], dim=1), y, d=torch.tensor([10]))

 
*** Profile stats marshalled to file '/var/folders/zc/6nw_r5b13ws3wfcpdxz6bzcw0000gn/T/tmpjl55c6fm'.
Opening SnakeViz in a new tab...
snakeviz web server started on 127.0.0.1:8080; enter Ctrl-C to exit
http://127.0.0.1:8080/snakeviz/%2Fvar%2Ffolders%2Fzc%2F6nw_r5b13ws3wfcpdxz6bzcw0000gn%2FT%2Ftmpjl55c6fm
